<a href="https://colab.research.google.com/github/saurabfaruque/pythonintro/blob/main/traveling_covid_abstract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
def read_excel_data():

    # Specify the columns we want to read
    cols = [f"t{i}_{var}" for var in ['cesd_tot', 'lonely_tot', 'anx_fin', 'exerc'] for i in range(1, 13)] #SF variables edited

    # Read the Excel file into a DataFrame, only reading the required columns
    #df = pd.read_excel("C19CS_only100.xlsx", usecols=cols)
    df = pd.read_excel("C19CS_Data_Numeric.xlsx", usecols=cols)

    # Reshape each variable into a 2D array and store them in a list
    variables = ['cesd_tot', 'lonely_tot', 'anx_fin', 'exerc'] #SF variables edited
    matrices = []
    for var in variables:
        cols = [f"t{i}_{var}" for i in range(1, 13)]
        matrix = df[cols].values,
        matrices.append(matrix)
    return matrices
[cesd_tot, lonely_tot, anx_fin, exerc] = read_excel_data() #SF variables edited

original_matrices = [cesd_tot, lonely_tot, anx_fin, exerc] #SF variables edited

In [ ]:
import pandas as pd

def read_excel_data():
    # Specify the columns we want to read
    cols = ["t0_age", "t0_sex", "t0_race_exp", "t0_relat", "t0_liv_a", "t0_educsn", "t0_cov_self", "t0_hhist_dep", "t0_hhist_anx", "t0_drink","t0_cesd_fin"] #added cesd_fin (0=nondepressed 1=depressed)

    # Read the Excel file into a DataFrame, only reading the required columns
    #df = pd.read_excel("C19CS_only100.xlsx", usecols=cols)
    df = pd.read_excel("C19CS_Data_Numeric.xlsx", usecols=cols)
    # Reshape each variable into a 2D array and store them in a list
    matrix = df[cols].values,

    return matrix

M2 = read_excel_data()


In [ ]:
[cesd_tot, lonely_tot, anx_fin, exerc] = original_matrices #SF variables edited
matrix_names = ['cesd_tot_clean', 'lonely_tot_clean', 'anx_fin_clean', 'exerc_clean'] #SF variables edited


In [ ]:
# New code for displaying the sample sizes
original_sample_sizes = [m[0].shape[0] for m in [cesd_tot, lonely_tot, anx_fin, exerc]]

for i, size in enumerate(original_sample_sizes):
    print(f'Sample size for {matrix_names[i]} before cleaning: {size}')

can you write a clean_data function that takes a list of 2D matrices, formatted like cesd_tot or lonely_tot above, and returns a new list of 2D matrices where any row that has a NAN in any of the 2D matrices is removed from all the 2D matrices.  

In [ ]:
# remove where there are any NANs, it's a lot
import numpy as np
ms = []
nans = []
nan_indices = nans
new_matrices = []
for m in original_matrices:
        m2 = np.squeeze(m)
        nans = np.where(np.any(np.isnan(m2), axis=1))[0]
        nan_indices=np.rint(np.squeeze(np.union1d(nan_indices, nans))).astype(int)
for m in original_matrices:
        m2 = np.squeeze(m)
        clean_m2 = np.delete(m2, nan_indices, axis=0)
        new_matrices.append(clean_m2)

M2_cleaned = np.delete(np.squeeze(M2), nan_indices, axis=0)


In [ ]:
sample_sizes = [m.shape[0] for m in new_matrices]

for i, sample_size in enumerate(sample_sizes):
    print(f'Sample size for {matrix_names[i]} after removing NaNs: {sample_size}')

write python code that will take the matrices variable matrices = [cesd_tot_clean, lonely_tot_clean, anx_fin_clean, worry_clean, satisfy_clean] = clean_data(original_matrices) and will output a new_matrices variable that has the same dimensions and same number of values, except it is normalized using standard fischer Z transform to put all the numbers in uniform normal distribution within dimension for the whole group.

In [ ]:
import numpy as np
from scipy.stats import zscore

def normalize_matrices(matrices):
    normalized_matrices = []
    for matrix in matrices:
        normalized_matrix = zscore(matrix, axis=0)  # Standardize along columns
        normalized_matrices.append(normalized_matrix)
    return normalized_matrices

new2_matrices = normalize_matrices(new_matrices)


In [ ]:
[cesd_tot_clean, lonely_tot_clean, anx_fin_clean, exerc_clean] = new2_matrices #SF variables edited

FIND PRIMARY COMPONENTS OF CHANGE WITH KMEANS:
Instead of principal components that can be added together to explain the 5 vector change scores, I would like a larger set of components that best account for the variance independently.  For any time point we will be modeling with just the statistically closest components scaled to account for that change.  so I would like to have a lot of these independent components ordered by how much variance they account for in the data.  Can you do that?

ChatGPT
From your description, it sounds like you want to use a method such as k-means clustering to partition your dataset into a larger number of groups or "components", each representing a distinct pattern of changes in your variables. Each cluster centroid would then be a 5-dimensional vector representing a mode of behavior, and each data point could be represented as the centroid of the cluster to which it's closest.

This is a different approach from PCA, which finds a smaller number of components that can be linearly combined to approximate the original dataset. Instead, k-means finds a set of representative points (the centroids) that partition the dataset into distinct clusters.

Here is how you might do this with the k-means algorithm in the Scikit-Learn library:  (but first a cell getting the data structured correctly)

...also write a python command that will a return 2D array output_array with array size num_subjects x time_points.  It will take as input change_array_3d which is a 3D array with array size num_subjects x num_time_points x num_dimensions. the command should iterate through the subjects in change_array_3d and fill the new 2D array  with the (11,) array output by use_viterbi using the 2D num_time_points x num_dimensions indexed in the iteration of subjects.

In [ ]:
change_vectors = []

# Compute change vectors for each matrix and store it in the list
for matrix in new2_matrices:
    # Compute the difference along the time axis (columns)
    change_vector = np.diff(matrix, axis=1)
    # Append the change_vector to the list
    change_vectors.append(change_vector)

#  ... came from somewhere else.   we may need the full group to get the k means better.  change_vector_matrix = change_vector(matrices)
# Stack the change_vectors along the new axis (axis=2) to create a 3D array
# This will have shape (num_subjects, num_timepoints, num_matrices)
change_array_3d = np.stack(change_vectors, axis=2)

# Reshape the 3D array into a 2D matrix
# This will have shape (num_subjects * num_timepoints, num_matrices)
change_vector_matrix = change_array_3d.reshape(-1, change_array_3d.shape[-1])

def describe_matrix(matrix):
    print(f"Matrix Shape: {matrix.shape}")
    print(f"Matrix Size: {matrix.size}")
    print(f"Matrix Datatype: {matrix.dtype}")
    print(f"Minimum Value in Matrix: {matrix.min()}")
    print(f"Maximum Value in Matrix: {matrix.max()}")
    print(f"Average Value in Matrix: {matrix.mean()}")
    print(f"Standard Deviation of Values in Matrix: {matrix.std()}")

#
# set some vars that get used later
num_subjects = change_vector.shape[0]
num_timepoints = change_vector.shape[1]
subject_id = np.repeat(np.arange(1, num_subjects+1), num_timepoints)

# Call a function to look at the change matrix
describe_matrix(change_vector_matrix)

In [ ]:
from sklearn.cluster import KMeans

# Determine the number of clusters you want
num_clusters = 5 # let's try

# Create the k-means object
kmeans = KMeans(n_clusters=num_clusters, random_state=0)

# Fit the k-means object to the data
kmeans.fit(change_vector_matrix)

# The cluster centers are your components
components = kmeans.cluster_centers_

# You can find the closest component to each data point
labels = kmeans.predict(change_vector_matrix)



CREATE TRANSITION ARRAY AND LIKELIHOOD MATRIX, the likelihood matrix is what the HMM code expects, and it is the traditional transition matrix of HMM.  the tranisition array here is all the data and likelihood is the summarized.  Sorry for the added confusion. Chatgpt prompt: write a function called create_transition_matrix.  This function will take as input subject_id and labels.  These vectors have the same number of rows (timepts-1)*num_subjects.  This new matrix will have fewer rows.  It will have (timepts-2)*num_subjects. The first column in the new matrix should be labels with the final timept for each subject removed.  The second column should be the labels vector with the first row of each subject removed and then shifted up (i.e., what was row 2 is now row 1)

In [ ]:
def create_transition_matrix(subject_id, labels):
    # unique subjects
    unique_subjects = np.unique(subject_id)

    # create empty list to store rows of new matrix
    transition_matrix = []

    # iterate over subjects
    for subj in unique_subjects:
        # get indices for the current subject
        subj_indices = np.where(subject_id == subj)[0]

        # get the labels for the current subject
        subj_labels = labels[subj_indices]

        # remove the last label for the current subject
        col1 = subj_labels[:-1]

        # remove the first label for the current subject
        col2 = subj_labels[1:]

        # append to the transition_matrix
        transition_matrix.extend(list(zip(col1, col2)))

    # convert list of tuples to numpy array
    transition_matrix = np.array(transition_matrix)

    return transition_matrix

transition_matrix= create_transition_matrix(subject_id,labels)

Write a function that takes as input the transition matrix (transition_matrix) from the create_transition_matrix and ouputs a 2D matrix, for the transition likelihoods of from the label in the row to the label in the col.  The likelihhods can be calculated from the frequencies of the labels appearing in col 1 and col 2 in the transition_matrix.  

In [ ]:
import numpy as np
from collections import Counter

def calculate_transition_likelihoods(transition_matrix):
    # Number of unique labels (clusters)
    num_labels = int(np.max(transition_matrix)) + 1

    # Initialize the likelihood matrix with zeros
    likelihood_matrix = np.zeros((num_labels, num_labels))

    # Count transitions from one state to another
    transition_counts = Counter(map(tuple, transition_matrix))

    # Fill in the likelihood matrix
    for (current_state, next_state), count in transition_counts.items():
        likelihood_matrix[int(current_state), int(next_state)] = count

    # Normalize rows to represent probabilities
    row_sums = likelihood_matrix.sum(axis=1, keepdims=True)
    likelihood_matrix = likelihood_matrix / row_sums

    return likelihood_matrix

likelihoods = calculate_transition_likelihoods(transition_matrix)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans

def feature_values_in_clusters(k_means, feature_indices, feature_names):
    n_clusters = k_means.n_clusters
    feature_values = k_means.cluster_centers_

    plt.figure(figsize=(20, 8))
    for i in range(n_clusters):
        plt.subplot(n_clusters//2 + n_clusters%2, 2, i+1)
        plt.barh(range(len(feature_indices)), feature_values[i, feature_indices], color='b', align='center')
        plt.yticks(range(len(feature_indices)), [feature_names[idx] for idx in feature_indices])
        plt.xlabel('Feature Value')
        plt.title(f'Cluster {i+1}')
    plt.tight_layout()
    plt.show()

# Select features by indices
feature_indices = [0, 1, 2, 3]  ##SF variables edited, removed the 4 # replace with your specific feature indices

# Call the function
feature_values_in_clusters(kmeans, feature_indices, matrix_names)

# Now let's print out the cluster centers
for i, center in enumerate(kmeans.cluster_centers_):
    print(f"Cluster {i+1}:")
    print(f"  cesd_tot_clean: {center[0]:.2f}")
    print(f"  lonely_tot_clean: {center[1]:.2f}")
    print(f"  anx_fin_clean: {center[2]:.2f}")
    print(f"  exerc_clean: {center[3]:.2f}") #SF variables edited
    print()


In [ ]:
#SF added, probability of each cluster being the first

def calculate_initial_probabilities(subject_id, labels):
    # Get the unique subjects
    unique_subjects = np.unique(subject_id)

    # Create an array to hold the count of each cluster appearing as the first state
    initial_state_counts = np.zeros(num_clusters)  # Assuming num_clusters is the number of clusters

    # Loop over each unique subject to find the first cluster label for that subject
    for subj in unique_subjects:
        # Get indices for the current subject
        subj_indices = np.where(subject_id == subj)[0]

        # Find the first label (cluster) for the current subject
        first_label = labels[subj_indices[0]]

        # Update the count for this label (cluster)
        initial_state_counts[first_label] += 1

    # Normalize the counts to get probabilities
    total_subjects = len(unique_subjects)
    initial_probabilities = initial_state_counts / total_subjects

    return initial_probabilities

# Calculate the initial probabilities
initial_probabilities = calculate_initial_probabilities(subject_id, labels)

print("Probability of each cluster being the first one [0,1,2,3,4]:", initial_probabilities)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn import hmm
import time
n_states = 5

def use_viterbi(observations):
    model = hmm.GaussianHMM(n_components=n_states,  covariance_type="diag")

    model.initial_probabilities = np.array([0.17496962, 0.2235723  ,0.09842041 ,0.41919806 ,0.08383961])  #SF edit, previously 1/5 each... updated based on the output of the block above "Probability of each cluster being the first one:"
    model.startprob_ = np.array([0.17496962, 0.2235723  ,0.09842041 ,0.41919806 ,0.08383961])  #SF edit, previously 1/5 each... updated based on the output of the block above "Probability of each cluster being the first one:"
    model.transmat_ = likelihoods
    model.means_ = components
    model.covars_ = np.ones((5, 4)) #SF edited, previously: model.covars_ = np.ones((5, 5))
    model.check_ = True
    model.sklearn_is_fitted_ = True
    predicted_states = model.predict(observations)
    return  predicted_states

def generate_output_array(change_array_3d):
    # Get the number of subjects and time points from the input array
    num_subjects, num_time_points, num_dimensions = change_array_3d.shape

    # Initialize an empty output array
    output_array = np.empty((num_subjects, num_time_points))

    # Iterate through the subjects
    for i in range(num_subjects):
        # Get the data for the current subject
        subject_data = change_array_3d[i]

        # Use the use_viterbi function to generate the output for this subject
        output_array[i] = use_viterbi(subject_data)

    return output_array

Covid_state_transitions = generate_output_array(change_array_3d)

In [ ]:
import numpy as np

# Assuming covid_state_transitions is an ndarray of shape (121, 11)
# Generate a sample covid_state_transitions for demonstration

# Initialize an empty array of zeros for the histogram with shape (121, 5)
covid_histogram = np.zeros((823, 5), dtype=int)

# Loop through each row in covid_state_transitions
for row in range(0,823):
    Current_row = Covid_state_transitions[row,:]
    covid_histogram[row,0] = np.count_nonzero(Current_row == 0)
    covid_histogram[row,1] = np.count_nonzero(Current_row == 1)
    covid_histogram[row,2] = np.count_nonzero(Current_row == 2)
    covid_histogram[row,3] = np.count_nonzero(Current_row == 3)
    covid_histogram[row,4] = np.count_nonzero(Current_row == 4)


# covid_histogram should now have the count of occurrences of each number (0 to 4) for each row.
print("Covid Histogram:")
print(covid_histogram)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming you've already extracted the depressed and non-depressed sub-groups:
t0_cesd_fin = M2_cleaned[:, -1]  # assuming t0_cesd_fin is the last column in M2_cleaned
depressed_indices = np.where(t0_cesd_fin == 1)[0]
non_depressed_indices = np.where(t0_cesd_fin == 0)[0]

covid_histogram_depressed = covid_histogram[depressed_indices]
covid_histogram_non_depressed = covid_histogram[non_depressed_indices]

# Calculate the mean for each column in the two sub-groups:
mean_depressed = np.mean(covid_histogram_depressed, axis=0)
mean_non_depressed = np.mean(covid_histogram_non_depressed, axis=0)

# Plotting
columns = ['0', '1', '2', '3', '4']

plt.figure(figsize=(10, 6))

bar_width = 0.35
index = np.arange(len(columns))

bar1 = plt.bar(index, mean_depressed, bar_width, color='b', label='Depressed')
bar2 = plt.bar(index + bar_width, mean_non_depressed, bar_width, color='r', label='Non-Depressed')

plt.xlabel('Columns')
plt.ylabel('Mean Frequency')
plt.title('Comparison of Mean Frequencies Between Depressed and Non-Depressed')
plt.xticks(index + bar_width / 2, columns)  # Positioning the labels in the middle of the combined bars
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

# Assuming you've already extracted the depressed and non-depressed sub-groups:
t0_cesd_fin = M2_cleaned[:, -1]  # assuming t0_cesd_fin is the last column in M2_cleaned
depressed_indices = np.where(t0_cesd_fin == 1)[0]
non_depressed_indices = np.where(t0_cesd_fin == 0)[0]

covid_histogram_depressed = covid_histogram[depressed_indices]
covid_histogram_non_depressed = covid_histogram[non_depressed_indices]

# Calculate the mean, standard deviation, and sample size for each column in the two sub-groups:
mean_depressed = np.mean(covid_histogram_depressed, axis=0)
std_depressed = np.std(covid_histogram_depressed, axis=0, ddof=1)
n_depressed = len(depressed_indices)

mean_non_depressed = np.mean(covid_histogram_non_depressed, axis=0)
std_non_depressed = np.std(covid_histogram_non_depressed, axis=0, ddof=1)
n_non_depressed = len(non_depressed_indices)

# Display means, stds, and sample sizes
columns = ['0', '1', '2', '3', '4']
for i, col in enumerate(columns):
    print(f"Column {col}:")
    print(f"  Depressed - Mean: {mean_depressed[i]:.2f}, Std: {std_depressed[i]:.2f}, N: {n_depressed}")
    print(f"  Non-Depressed - Mean: {mean_non_depressed[i]:.2f}, Std: {std_non_depressed[i]:.2f}, N: {n_non_depressed}")

    # Conduct t-test
    t_stat, p_val = ttest_ind(covid_histogram_depressed[:, i], covid_histogram_non_depressed[:, i], equal_var=False)
    print(f"  t-test: t = {t_stat:.2f}, p = {p_val:.4f}\n")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming you've already extracted the demographic info:
t0_cesd_fin = M2_cleaned[:, -1]  # assuming t0_cesd_fin is the last column in M2_cleaned
t0_sex = M2_cleaned[:, 1]  # assuming t0_sex is the second column in M2_cleaned

columns = ['0', '1', '2', '3', '4']

# Define a function to generate the plots
def plot_by_group(depressed_data, non_depressed_data, group_name):
    mean_depressed = np.mean(depressed_data, axis=0)
    mean_non_depressed = np.mean(non_depressed_data, axis=0)

    plt.figure(figsize=(10, 6))
    bar_width = 0.35
    index = np.arange(len(columns))

    plt.bar(index, mean_depressed, bar_width, color='b', label='Depressed')
    plt.bar(index + bar_width, mean_non_depressed, bar_width, color='r', label='Non-Depressed')

    plt.xlabel('Columns')
    plt.ylabel('Mean Frequency')
    plt.title(f'Comparison of Mean Frequencies Between Depressed and Non-Depressed ({group_name})')
    plt.xticks(index + bar_width / 2, columns)
    plt.legend()

    plt.tight_layout()
    plt.show()

# Stratify by gender
for gender in [0, 1]:  # assuming 0 and 1 are the codes for the two genders
    gender_indices = np.where(t0_sex == gender)[0]

    depressed_by_gender = covid_histogram[np.intersect1d(depressed_indices, gender_indices)]
    non_depressed_by_gender = covid_histogram[np.intersect1d(non_depressed_indices, gender_indices)]

    plot_by_group(depressed_by_gender, non_depressed_by_gender, f"Gender {gender}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

# Assuming you've already extracted the demographic info:
t0_cesd_fin = M2_cleaned[:, -1]
t0_sex = M2_cleaned[:, 1]

columns = ['0', '1', '2', '3', '4']

# Define a function to compute and display statistics
def compute_statistics(depressed_data, non_depressed_data, group_name):
    t_values = []
    p_values = []

    mean_depressed = np.mean(depressed_data, axis=0)
    std_depressed = np.std(depressed_data, axis=0)
    size_depressed = len(depressed_data)

    mean_non_depressed = np.mean(non_depressed_data, axis=0)
    std_non_depressed = np.std(non_depressed_data, axis=0)
    size_non_depressed = len(non_depressed_data)

    for i in range(depressed_data.shape[1]):
        t_val, p_val = ttest_ind(depressed_data[:, i], non_depressed_data[:, i])
        t_values.append(t_val)
        p_values.append(p_val)

    print(f"------ {group_name} ------")
    for col, t, p, mean_d, std_d, mean_nd, std_nd in zip(columns, t_values, p_values,
                                                         mean_depressed, std_depressed,
                                                         mean_non_depressed, std_non_depressed):
        print(f"Column {col}:")
        print(f"  Depressed: mean={mean_d:.2f}, std={std_d:.2f}, n={size_depressed}")
        print(f"  Non-Depressed: mean={mean_nd:.2f}, std={std_nd:.2f}, n={size_non_depressed}")
        print(f"  t={t:.2f}, p={p:.4f}")
    print()

# Stratify by gender and compute statistics
for gender in [0, 1]:
    gender_indices = np.where(t0_sex == gender)[0]

    depressed_by_gender = covid_histogram[np.intersect1d(depressed_indices, gender_indices)]
    non_depressed_by_gender = covid_histogram[np.intersect1d(non_depressed_indices, gender_indices)]

    compute_statistics(depressed_by_gender, non_depressed_by_gender, f"Gender {gender}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

# Assuming you've already extracted the demographic info:
t0_cesd_fin = M2_cleaned[:, -1]
t0_sex = M2_cleaned[:, 1]
columns = ['0', '1', '2', '3', '4']

# 1. Main Effect of Gender:
print("------ Main Effect of Gender ------")
men_indices = np.where(t0_sex == 0)[0]
women_indices = np.where(t0_sex == 1)[0]

men_data = covid_histogram[men_indices]
women_data = covid_histogram[women_indices]

for col in range(men_data.shape[1]):
    t_val, p_val = ttest_ind(men_data[:, col], women_data[:, col])
    print(f"Column {columns[col]}: t={t_val:.2f}, p={p_val:.4f}")

# 2. Interaction Effect of Depression x Gender:
print("\n------ Interaction Effect of Depression x Gender ------")
for gender, name in [(0, "Men"), (1, "Women")]:
    gender_indices = np.where(t0_sex == gender)[0]

    depressed_by_gender = covid_histogram[np.intersect1d(depressed_indices, gender_indices)]
    non_depressed_by_gender = covid_histogram[np.intersect1d(non_depressed_indices, gender_indices)]

    print(f"\nFor {name}:")
    for col in range(depressed_by_gender.shape[1]):
        t_val, p_val = ttest_ind(depressed_by_gender[:, col], non_depressed_by_gender[:, col])
        print(f"Column {columns[col]}: t={t_val:.2f}, p={p_val:.4f}")
